In [ ]:
from fastai.gen_doc.gen_notebooks import update_nb_metadata
# For updating jekyll metadata. You MUST reload notebook immediately after executing this cell for changes to save
update_nb_metadata('tabular.ipynb', title=None, summary="Data, transforms, models, and training for tabular/structured data")

# Tabular data

In [ ]:
%reload_ext autoreload
%autoreload 2

from fastai.gen_doc.nbdoc import *
from fastai.tabular import * 
from fastai import *
from fastai.docs import *
from fastai.models.tabular import *

[`tabular`](/tabular.html#tabular) contains all the necessary classes to deal with tabular data, across two modules:
- [`tabular.transform`](/text.transform.html#text.transform): defines the [`TabularTransform`](/tabular.transform.html#TabularTransform) class to help with preprocessing;
- [`tabular.data`](/text.data.html#text.data): defines the [`TabularDataset`](/tabular.data.html#TabularDataset) that handles that data, as well as the methods to quickly get a [`DataBunch`](/data.html#DataBunch).

To create a model, you'll need to use [`models.tabular`](/tabular.html#tabular). See below for and end-to-end example using all these modules.

## Preprocessing tabular data

Tabular data usually comes in the form of a delimited file (such as CSV) containing variables of different kinds: text/category, numbers, and perhaps some missing values. The example we'll work with in this section is a sample of the [adult dataset](https://archive.ics.uci.edu/ml/datasets/adult) which has some census information on individuals. We'll use it to train a model to predict whether salary is greater than \$50k or not.

In [ ]:
untar_adult()
ADULT_PATH

PosixPath('../data/adult_sample')

In [ ]:
df = pd.read_csv(ADULT_PATH/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


Here all the information that will form our input is in the 14 first columns, and the dependant variable is the last column. We will split our input between two types of variables: categoricals and continuous.
- Categorical variables will be replaced by a category, a unique id that identifies them, before passing through an embedding layer.
- Continuous variables will be normalized then directly fed to the model.

Another thing we need to handle are the missing values: our model isn't going to like receiving NaNs so we should remove them in a smart way. All of this preprocessing is done by [`TabularTransform`](/tabular.transform.html#TabularTransform) objects and [`TabularDataset`](/tabular.data.html#TabularDataset).

We can define a bunch of Transforms that will be applied to our variables. Here we transform all categorical variables into categories, and we replace missing values for continuous variables by the median of the corresponding column.

In [ ]:
tfms = [FillMissing, Categorify]

We split our data into training and validation sets.

In [ ]:
train_df, valid_df = df[:-2000].copy(),df[-2000:].copy()

First let's split our variables between categoricals and continuous (we can ignore the dependant variable at this stage). fastai will assume all variables that aren't dependent or categorical are continuous, unless we explicitly pass a list to the `cont_names` parameter when constructing our [`DataBunch`](/data.html#DataBunch).

In [ ]:
dep_var = '>=50k'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

Now we're ready to pass these details to [`tabular_data_from_df`](/tabular.data.html#tabular_data_from_df) to create the [`DataBunch`](/data.html#DataBunch) that we'll use for training.

In [ ]:
data = tabular_data_from_df(ADULT_PATH, train_df, valid_df, dep_var, tfms=tfms, cat_names=cat_names)
print(data.train_ds.cont_names)  # `cont_names` defaults to: set(df)-set(cat_names)-{dep_var}

['hours-per-week', 'fnlwgt', 'age', 'capital-gain', 'capital-loss', 'education-num']


We can grab a mini-batch of data and take a look (note that `to_np` here converts from pytorch tensor to numpy):

In [ ]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[ 5 10  3 11  1  5  2 40  1]
 [ 5 12  3  4  1  5  2 40  1]
 [ 5 16  3  2  1  5  2 40  1]
 [ 5  2  1 13  2  5  1 40  1]
 [ 5 12  3  7  1  5  2 40  1]]
[[ 0.7721204  -0.0401719   1.7117627  -0.14592563 -0.21676035  1.1458108 ]
 [ 0.28737482  1.320818    1.1259431  -0.14592563 -0.21676035 -0.42159945]
 [ 0.7721204  -1.2330674  -0.5582884  -0.14592563 -0.21676035 -0.02974687]
 [-0.03578891 -0.02159784  0.83303326 -0.14592563 -0.21676035 -1.2053045 ]
 [-0.03578891 -0.7227921  -0.19215111 -0.14592563 -0.21676035 -0.42159945]]
[1 1 0 0 0]


After being processed in [`TabularDataset`](/tabular.data.html#TabularDataset), the categorical variables are replaced by ids and the continuous variables are normalized. The codes corresponding to categorical variables are all put together, as are all the continuous variables.

## Defining a model

Once we have our data ready in a [`DataBunch`](/data.html#DataBunch), we just need to create a model to then define a [`Learner`](/basic_train.html#Learner) and start training. The fastai library has a flexible and powerful [`TabularModel`](/models.tabular.html#TabularModel) in [`models.tabular`](/tabular.html#tabular). To use that function, we just need to specify the embedding sizes for each of our categorical variables.

In [ ]:
model = get_tabular_learner(data, layers=[200,100], emb_szs={'native-country': 10})
learn = Learner(data, model, metrics=accuracy)
learn.fit_one_cycle(1, 1e-2)

Total time: 00:05
epoch  train loss  valid loss  accuracy
0      0.331778    0.346305    0.834000  (00:05)

